# Interactive t-SNE ROOTS dimensionality reduction (*datashaded*)

In [ ]:
import pandas as pd
import colorcet as cc
import holoviews as hv
import panel as pn

import warnings
warnings.filterwarnings(action="ignore")
import datashader as ds
from holoviews.operation.datashader import datashade, rasterize, dynspread
hv.extension('bokeh')


*Visualization of t-Distributed Stochastic Neighbor Embedding of ROOTS language Corpus.*

Trained data from Christopher Akiki (dataset `cakiki/roots-tsne-data` on Hugging Face 

A version of this notebook that runs color-mixing on the client-side can be found [here](./t_sne_roots.ipynb). Here are the key differences between the datashaded and rasterized versions:

* The rasterized version is color mixed in the browser, allowing enhanced interactivity (e.g. colorbars)
* The rasterized version returns an `ImageStack` while the datashaded version returns an `RGB`: the former retains the data statistics. 
* The datashaded version uses `dynspread` as `ImageStack` does not yet support spreading.



## Load the data

In [ ]:
tsne_embedding = pd.read_parquet('./data/train-00000-of-00001-9a434d9cf7fd233e.parquet')
df = pd.DataFrame(data=tsne_embedding, columns=['x','y','language'])
df.head()

## Datashade and add hover layer

Note that until client-side colormixing is supported, the `datashade` operation is used with an invisible hover layer overlaid on top.

In [ ]:
points = hv.Points(df, ['x','y'], ['language'])
op = datashade(points, color_key=cc.glasbey_light,
               aggregator=ds.by('language', ds.count())).opts(show_legend=False, bgcolor='black', frame_width=600, aspect=1)
hover_layer = rasterize(points, selector=ds.first("x"), x_sampling=2, y_sampling=2).opts(tools=["hover"], alpha=0)

### Render

In [ ]:
dynspread(op) * hover_layer

### Build a small dashboard with panel

In [ ]:
text = """
### Visualization of t-Distributed Stochastic Neighbor Embedding of ROOTS language Corpus. <br> Trained data from Christopher Akiki (dataset `cakiki/roots-tsne-data` on Hugging Face
"""

template = pn.template.FastListTemplate(
    title="t-SNE clustering of ROOTS language corpus"
)

template.main.append(
    pn.Column(
        pn.pane.Markdown(text),
        pn.Row(pn.HSpacer(), dynspread(op) * hover_layer, pn.HSpacer())
    )
)
template.servable();